In [16]:
# For Data Processing
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from PIL import Image, ImageEnhance

# For ML Models
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img

# Miscellaneous
from tqdm import tqdm
import os
import random

In [17]:
train_directory = './data/train/'
test_directory = './data/test/'

training_paths = []
training_labels = []
testing_paths = []
testing_labels = []
unique_labels = []

for label in os.listdir(train_directory):
    if(label[0] != '.'):
        unique_labels.append(label)
        for image in os.listdir(train_directory + label):
            training_paths.append(train_directory + label + '/' + image)
            training_labels.append(label)
        
for label in os.listdir(test_directory):
    if(label[0] != '.'):
        for image in os.listdir(test_directory + label):
            testing_paths.append(test_directory + label + '/' + image)
            testing_labels.append(label)
        
training_paths, training_labels = shuffle(training_paths, training_labels)
testing_paths, testing_labels = shuffle(testing_paths, testing_labels)
    

In [22]:
# define functions
pic_size = 128

def pic_enhance(pic):
    pic = Image.fromarray(np.uint8(pic))
    # random brightness and constrast between 80% - 120%
    pic = ImageEnhance.Brightness(pic).enhance(random.uniform(0.8, 1.2))
    pic = ImageEnhance.Contrast(pic).enhance(random.uniform(0.8, 1.2))
    # change pic to 255 size for VGG
    pic = np.array(pic) / 255.0
    return pic

def get_pics(paths):
    pics = []
    for p in paths:
        pic = load_img(p, target_size=(pic_size, pic_size))
        pic = pic_enhance(pic)
        pics.append(pic)
    return np.array(pics)

def normalize(labels):
    label_idx = []
    for l in labels:
        label_idx.append(unique_labels.index(l))
    return np.array(label_idx)

def denormalize(labels):
    label_vals = []
    for l in labels:
        label_vals.append(unique_labels[l])
    return np.array(label_vals)

def create_datagen(paths, labels, batch_size, epochs):
    for e in range(epochs):
        for i in range(0, len(paths), batch_size):
            batch = paths[i : i + batch_size]
            batch_pics = get_pics(batch)
            batch_labels = labels[i : i + batch_size]
            batch_labels = normalize(batch_labels)
            yield batch_pics, batch_labels

In [19]:
# create VGG16 model using imagenet for transfer learning
base = VGG16(input_shape = (pic_size, pic_size, 3), include_top = False, weights = 'imagenet')

# make all layers of VGG16 base model untrainable to retain imagenet training weights
for layer in base.layers:
    layer.trainable = False
    
# set the last vgg block, the top layer of the model, to trainable. This will be trained on our MRI images
base.layers[-2].trainable = True
base.layers[-3].trainable = True
base.layers[-4].trainable = True

model = Sequential()
model.add(Input(shape = (pic_size, pic_size, 3)))
model.add(base)
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(len(unique_labels), activation = 'softmax'))

model.summary()

model.compile(optimizer = Adam(learning_rate = 0.0001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy', 'sparse_categorical_accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_6 (Dropout)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               1048704   
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 258       
                                                                 
Total params: 15,763,650
Trainable params: 8,128,386
N

In [20]:
batch_size = 20
steps = int(len(training_paths) / batch_size)
epochs = 4
history = model.fit(create_datagen(training_paths, training_labels, batch_size, epochs), epochs = epochs, steps_per_epoch = steps)

Epoch 1/4


2022-11-16 18:27:43.962244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


285/285 [==============================] - 22s 76ms/step - loss: 0.1203 - accuracy: 0.9619 - sparse_categorical_accuracy: 0.9619
Epoch 2/4
285/285 [==============================] - 22s 76ms/step - loss: 0.0423 - accuracy: 0.9845 - sparse_categorical_accuracy: 0.9845
Epoch 3/4
285/285 [==============================] - 22s 77ms/step - loss: 0.0279 - accuracy: 0.9900 - sparse_categorical_accuracy: 0.9900
Epoch 4/4
285/285 [==============================] - 22s 77ms/step - loss: 0.0229 - accuracy: 0.9926 - sparse_categorical_accuracy: 0.9926


In [23]:
batch_size = 32
steps = int(len(testing_paths) / batch_size)
y_prediction = []
y_ans = []
for x, y in tqdm(create_datagen(testing_paths, testing_labels, batch_size, epochs), total = steps):
    prediction = model.predict(x)
    prediction = np.argmax(prediction, axis = -1)
    for label in denormalize(prediction):
        y_prediction.append(label)
    for label in denormalize(y):
        y_ans.append(label)
        
print(classification_report(y_ans, y_prediction))

  0%|          | 0/40 [00:00<?, ?it/s]

1/1 [==============================] - 0s 16ms/step


  2%|▎         | 1/40 [00:00<00:07,  5.02it/s]

1/1 [==============================] - 0s 15ms/step


  5%|▌         | 2/40 [00:00<00:06,  5.64it/s]

...

1/1 [==============================] - 0s 16ms/step


163it [00:25,  6.65it/s]

1/1 [==============================] - 0s 17ms/step


164it [00:25,  6.47it/s]

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3624
           1       0.98      0.99      0.99      1620

    accuracy                           0.99      5244
   macro avg       0.99      0.99      0.99      5244
weighted avg       0.99      0.99      0.99      5244



In [24]:
model.save('VGG16_trained_model.h5')